### 7. Rising Temperature Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday). Return the result table in any order.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *  # Import the function
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import col, explode, array_repeat
from pyspark.sql.window import Window
import json
spark = SparkSession.builder.getOrCreate()

In [2]:
# Define the schema for the "Weather" table
weather_schema = StructType([
StructField("id", IntegerType(), True),
StructField("recordDate", StringType(), True),
StructField("temperature", IntegerType(), True)
])
# Define data for the "Weather" table
weather_data = [
(1, '2015-01-01', 10),
(2, '2015-01-02', 25),
(3, '2015-01-03', 20),
(4, '2015-01-04', 30)
]
# Create the "Weather" DataFrame
weather_df = spark.createDataFrame(weather_data, weather_schema)
weather_df.show()

+---+----------+-----------+
| id|recordDate|temperature|
+---+----------+-----------+
|  1|2015-01-01|         10|
|  2|2015-01-02|         25|
|  3|2015-01-03|         20|
|  4|2015-01-04|         30|
+---+----------+-----------+



In [3]:
w = Window.orderBy('recordDate')

lag_df = weather_df.withColumn("prev_day_temp", lag(col('temperature')).over(w))
lag_df.show()

+---+----------+-----------+-------------+
| id|recordDate|temperature|prev_day_temp|
+---+----------+-----------+-------------+
|  1|2015-01-01|         10|         NULL|
|  2|2015-01-02|         25|           10|
|  3|2015-01-03|         20|           25|
|  4|2015-01-04|         30|           20|
+---+----------+-----------+-------------+



In [4]:
filtered_df = lag_df.filter(col('temperature') > col('prev_day_temp'))
filtered_df.show()


+---+----------+-----------+-------------+
| id|recordDate|temperature|prev_day_temp|
+---+----------+-----------+-------------+
|  2|2015-01-02|         25|           10|
|  4|2015-01-04|         30|           20|
+---+----------+-----------+-------------+

